# Введение в нейронные сети
## Задание 1

### Импортируем все необходимые библиотеки

In [1]:
"""
Разпознование цифр Mnist с помощью Keras (tensorflow)
"""
from typing import Tuple

import numpy as np
import cv2

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.src.utils import to_categorical

from keras.datasets import mnist

np.random.seed(123)


### Скачиваем датасет mnist

In [2]:

def get_train_and_test_data() -> Tuple[Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray]]:
    """
    Загрузка и подготовка датасета Mnist
    :return: (x_train, y_train), (x_test, y_test)
    """

    # загружаем датасет
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train = x_train.astype(dtype=float)
    x_test = x_test.astype(dtype=float)

    x_train /= 255
    x_test /= 255

    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    return (x_train, y_train), (x_test, y_test)

**Создаём модель 1**

In [3]:

def create_model():
    """
    Создание простой модели
    :return:
    """
    new_model = Sequential()

    new_model.add(Flatten())
    new_model.add(Dense(128))
    new_model.add(Activation(tf.nn.relu))
    new_model.add(Dense(64))
    new_model.add(Activation(tf.nn.relu))
    new_model.add(Dense(10))
    new_model.add(Activation(tf.nn.sigmoid))

    return new_model

**Создаём модель 2**

In [4]:
def create_model_conv2d():
    """
    Создание более сложной/тяжелой модели
    :return:
    """
    new_model = Sequential()

    new_model.add(Convolution2D(32, (3, 3), input_shape=(28, 28, 1)))
    new_model.add(Activation(tf.nn.relu))
    new_model.add(Convolution2D(32, (3, 3)))
    new_model.add(Activation(tf.nn.relu))
    new_model.add(MaxPooling2D(pool_size=(2, 2)))
    new_model.add(Dropout(0.25))

    new_model.add(Flatten())
    new_model.add(Dense(128))
    new_model.add(Activation(tf.nn.relu))
    new_model.add(Dropout(0.5))
    new_model.add(Dense(10))
    new_model.add(Activation(tf.nn.softmax))

    return new_model

**Обучение модели**

In [35]:

def train_model(model_to_train, x_train: np.ndarray, y_train: np.ndarray, batch_size=32, epochs=10, verbose=1) -> None:
    """
    Обучение модели
    :param model_to_train: Модель
    :param x_train:
    :param y_train:
    :param batch_size:
    :param epochs:
    :param verbose:
    :return:
    """

    # выбираем оптимизатор
    optimizer = tf.keras.optimizers.SGD()

    # функция потерь
    loss = tf.keras.losses.CategoricalCrossentropy()

    # подготовка модели и задание метрик для обучения
    model_to_train.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

    # обучение модели
    model_to_train.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose)

    # summary модели

    model_to_train.summary()



**Проверка модели**

In [6]:
def test_model(model_to_test, x_test: np.ndarray, y_test: np.ndarray, show_image=True) -> None:
    """
    Проверка модели на тестовой выбборке
    :param model_to_test:
    :param x_test:
    :param y_test:
    :param show_image: Показывать картинку
    :return:
    """

    # выбираем несколько картиток случайным образом
    for i in np.random.choice(np.arange(0, len(y_test)), size=(20,)):
        # распознование
        probs = model_to_test.predict(x_test[np.newaxis, i], verbose=0)
        # выбираем с макс. вероятностью
        prediction = probs.argmax(axis=1)

        print(f"{i}, Actual digit is {y_test[i].argmax()}, predicted {prediction[0]}")

        if show_image:
            image = (x_test[i] * 255).reshape((28, 28)).astype("uint8")

            cv2.imshow("Digit", image)
            cv2.waitKey(0)

**Модель №1**

In [41]:
def model_1():
    print("Test simple model")
    model = create_model()

    (X_train, y_train), (X_test, y_test) = get_train_and_test_data()

    train_model(model_to_train=model, x_train=X_train, y_train=y_train, epochs=50)
    test_model(model_to_test=model, x_test=X_test, y_test=y_test, show_image=False)

In [42]:
model_1()

Test simple model
Epoch 1/50
1875/1875 [==============================] - 7s 3ms/step - loss: 0.7908 - accuracy: 0.7841
Epoch 2/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3123 - accuracy: 0.9106
Epoch 3/50
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2562 - accuracy: 0.9272
Epoch 4/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2211 - accuracy: 0.9377
Epoch 5/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1936 - accuracy: 0.9447
Epoch 6/50
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1716 - accuracy: 0.9507
Epoch 7/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1540 - accuracy: 0.9556
Epoch 8/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1396 - accuracy: 0.9601
Epoch 9/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1279 - accuracy: 0.9633
Epoch 10/50
1875/1875 [==============================] - 5s 3ms/

**Модель №2**

In [39]:
def model_2():
    print("Test model with conv2d layers")

    model = create_model_conv2d()

    (X_train, y_train), (X_test, y_test) = get_train_and_test_data()

    train_model(model_to_train=model, x_train=X_train, y_train=y_train, epochs=50)
    test_model(model_to_test=model, x_test=X_test, y_test=y_test, show_image=False)

In [40]:
model_2()

Test model with conv2d layers
Epoch 1/50
1875/1875 [==============================] - 9s 4ms/step - loss: 0.6014 - accuracy: 0.8095
Epoch 2/50
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3094 - accuracy: 0.9072
Epoch 3/50
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2386 - accuracy: 0.9280
Epoch 4/50
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1947 - accuracy: 0.9411
Epoch 5/50
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1606 - accuracy: 0.9518
Epoch 6/50
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1437 - accuracy: 0.9572
Epoch 7/50
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1264 - accuracy: 0.9618
Epoch 8/50
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1133 - accuracy: 0.9662
Epoch 9/50
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1061 - accuracy: 0.9680
Epoch 10/50
1875/1875 [=============================

### Сравниваем несколько архитектур:

1)

2)

3)

## Выводы:

т